In [63]:
import pandas as pd
import json
import openai
import secret

In [64]:
openai.api_key = secret.apiKey

In [35]:
development = pd.read_csv("../data/development.csv")

In [37]:
gpt3Train = pd.DataFrame({"prompt": development["tweet"], "completion": development["ideology_multiclass"]})
gpt3Train["prompt"] = gpt3Train["prompt"] + "->"
gpt3Train["completion"] = gpt3Train["completion"] + '\n'

In [42]:
obj = json.loads(gpt3Train.sample(1000).to_json(orient='records'))

In [43]:
with open('gpt3-train.jsonl', 'w') as output:
    for entry in obj:
        json.dump(entry, output)
        output.write("\n")

In [53]:
fileid = openai.File.create(
    file=open('gpt3-train.jsonl', 'rb'),
    purpose='fine-tune'
)

In [56]:
openai.FineTune.create(training_file=fileid["id"])

<FineTune fine-tune id=ft-8isr7MOodVSaIavj4zlAZHoX at 0x144950900> JSON: {
  "created_at": 1681166198,
  "events": [
    {
      "created_at": 1681166198,
      "level": "info",
      "message": "Created fine-tune: ft-8isr7MOodVSaIavj4zlAZHoX",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-8isr7MOodVSaIavj4zlAZHoX",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-L1lljxbCYtTHvDgsjfIcJcS5",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 242709,
      "created_at": 1681166122,
      "filename": "file",
      "id": "file-YYP7VJQyA2mv2xOJbL0RkyXv",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1681166198,
  "validation_files": []
}

In [67]:
openai.FineTune.list_events(id="ft-8isr7MOodVSaIavj4zlAZHoX")

<OpenAIObject list at 0x1351e0090> JSON: {
  "data": [
    {
      "created_at": 1681166198,
      "level": "info",
      "message": "Created fine-tune: ft-8isr7MOodVSaIavj4zlAZHoX",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1681166282,
      "level": "info",
      "message": "Fine-tune costs $0.81",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1681166282,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1681166888,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1681167187,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1681167425,
      "level": "info",
      "message": "Completed epoch 2/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1681167662,
      "level

In [61]:
openai.FineTune.list()

<OpenAIObject list at 0x1351da860> JSON: {
  "data": [
    {
      "created_at": 1681166198,
      "fine_tuned_model": null,
      "hyperparams": {
        "batch_size": 2,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-8isr7MOodVSaIavj4zlAZHoX",
      "model": "curie",
      "object": "fine-tune",
      "organization_id": "org-L1lljxbCYtTHvDgsjfIcJcS5",
      "result_files": [],
      "status": "pending",
      "training_files": [
        {
          "bytes": 242709,
          "created_at": 1681166122,
          "filename": "file",
          "id": "file-YYP7VJQyA2mv2xOJbL0RkyXv",
          "object": "file",
          "purpose": "fine-tune",
          "status": "processed",
          "status_details": null
        }
      ],
      "updated_at": 1681166282,
      "validation_files": []
    }
  ],
  "object": "list"
}

In [70]:
info = openai.FineTune.retrieve('ft-8isr7MOodVSaIavj4zlAZHoX')

In [80]:
id_curie_train_1000 = 'curie:ft-personal-2023-04-10-23-05-20'

In [83]:
prediction = openai.Completion.create(
    model=id_curie_train_1000,
    prompt="No seremos obstáculo, pero estos presupuestos no llevarán la firma de [POLITICAL_PARTY].->"
)

In [84]:
prediction

<OpenAIObject text_completion id=cmpl-74CKy2hZNtYZaKoeyVFrf0ddLzIYy at 0x1351daf90> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "moderate_right\n\n. moderate_left\n\nSe respeta a to"
    }
  ],
  "created": 1681235092,
  "id": "cmpl-74CKy2hZNtYZaKoeyVFrf0ddLzIYy",
  "model": "curie:ft-personal-2023-04-10-23-05-20",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 36,
    "total_tokens": 52
  }
}